# Data Clean Up

Looking through the data I have, I need to resolve 'duplicates' and null values, evaluate if they are true duplicates/nulls. I also noticed that the featured column does not list the actual artist that was collaborating on the track, instead has the html tags, I will need to extract the artist names from this section and re-place the current value. The same goes for media. I want to extract the Spotify ID in-case I want to extract audio features. 

<p> I will need to create a target column based on whether the artists sing their own names/labels in the track. I will need to create an 'Alias' column; Akon will almost always sing 'Konvict' in his tracks, Nicki's 'Young Money', Lil' Wayne's 'Weezy' etc.

### Imports

In [1]:
import pandas as pd
import numpy as np
import regex as re

In [2]:
df = pd.read_csv('./data/gathering2_lyrics.csv')

In [3]:
df.head()

,artist,featured,title,media,release_year,lyrics
0,Jason Derulo,"[{'api_path': '/artists/25005', 'header_image_...",Swalla,"[{'provider': 'youtube', 'start': 0, 'type': '...",2017-02-24,[Intro: Nicki Minaj]\nDrank\nYoung Money\n\n[V...
1,Jason Derulo,"[{'api_path': '/artists/14325', 'header_image_...",Talk Dirty,"[{'provider': 'youtube', 'start': 0, 'type': '...",2013-08-02,"[Intro: Jason Derulo & Rie Abe]\n(Jason, haha\..."
2,Jason Derulo,"[{'api_path': '/artists/46', 'header_image_url...",Wiggle,[{'native_uri': 'spotify:track:2sLwPnIP3CUVmIu...,2014-06-06,"[Intro: Jason Derulo & Snoop Dogg]\nAyo, Jason..."
3,Jason Derulo,[],Trumpets,[{'native_uri': 'spotify:track:5KONnBIQ9LqCxye...,2013-11-07,[Chorus]\nEvery time that you get undressed\nI...
4,Jason Derulo,"[{'api_path': '/artists/1583', 'header_image_u...",Tip Toe,[{'native_uri': 'spotify:track:2z4pcBLQXF2BXKF...,2017-11-10,[Intro: Jason Derulo & Soaky Siren]\nDerulo\nW...


Checking the describe of the data and adding a column for nulls for quick viewing

In [4]:
desc = df.describe() # assign describe to variable
add_null = pd.concat([df.isnull().sum().rename('Nulls'),desc.T],axis=1)

In [5]:
add_null

,Nulls,count,unique,top,freq
artist,0,11890,130,Bruno Mars,100
featured,0,11890,1899,[],8784
title,0,11890,11000,Intro,17
media,0,11890,10540,[],1340
release_year,1920,9970,2786,2016-03-25,42
lyrics,3,11887,11640,[Instrumental],62


## Cleanup

For the missing year values, It is in a yyy-mm-dd format, I only want the year, and will try to fill the values that I can as I fill the lyrics

In [6]:
df['release_year'] = df['release_year'].str[:4] # splitting the strings in release_year value to only give me the year

#### Checking Nulls

In [7]:
df.loc[df['lyrics'].isnull()]
# Elastic Heart already exhists
# Yellow Flicker Beat already exhists
# Pharrell Burger haha he has his own Signature Burger in Tokyo...

,artist,featured,title,media,release_year,lyrics
2052,Sia,"[{'api_path': '/artists/49361', 'header_image_...",Elastic Heart (Video Breakdown),"[{'provider': 'youtube', 'start': 0, 'type': '...",NaN,NaN
2378,Lorde,[],“Yellow Flicker Beat” Single Art,[],2014,NaN
5321,Pharrell Williams,[],Pharrell Burger,[],2014,NaN


Dropping the null values in lyrics

In [8]:
df.drop(df.loc[df['lyrics'].isnull()].index, inplace=True)

In [9]:
df.shape

(11887, 6)

In [10]:
df[df.duplicated('lyrics', keep = False)]

,artist,featured,title,media,release_year,lyrics
95,Jason Derulo,[],Red Card,[],1996,\n Lyrics for this song h...
155,CAKE,[],Never Gonna Give You Up,[],2007,You look so good to me right now...\nYou reall...
174,CAKE,[],Bound Away,[],2011,I'm an unknown individual in an unattended car...
179,CAKE,[],Teenage Pregnancy,[],2011,[Instrumental]
184,CAKE,[],Conroy,[],2007,[Instrumental]
...,...,...,...,...,...,...
11561,Ava Max,[],My Way (Shew Remix),"[{'attribution': 'iamshew', 'provider': 'sound...",2018,"[Verse 1]\nMy momma use to say\n""Baby make me ..."
11563,Ava Max,[],Dream Away,[],2006,No Lyrics Available Yet
11570,Ava Max,[],Treat Me Like a Lady,[],2006,No Lyrics Available Yet
11571,Ava Max,[],Head & Heart,[],NaN,\n Lyrics for this song h...


Of the 332 duplicates, there are 63 songs where the lyrics of the song have not yet been release or are unavailable. Additionally there are 69 rows where the lyrics are Instrumental. This is due to some artists like Christina Perri who also do lullabies, MGMT, Diplo, who do electronic music and songs that are Intros to albums. 
The rest of the tracks that were marked duplicates were true duplicates. Fisherspooner had 10 songs that were the same, because of different Remixes, but the lyrics are all the same
<br> Then dropping another 104 rows where the lyrics are less than 150 characters in length, grabbing some odd bits that were pulled </br> 
<br> I decided to drop BLACKPINK because majority of the songs were duplicates and in Korean script .Initially chose them in my list of artists because I knew they shouted their name in the beginning of the songs, however even the songs were the same the lyrics were written differently (dialect) and would not be flagged as duplicates. I noticed that of the 88 songs that were pulled only about 12 of them were not duplicates.

In [11]:
df.drop_duplicates(subset='lyrics', keep="first", inplace=True) # Keeping first instance and dropping the rest

In [12]:
df.drop(df.loc[df['lyrics'].str.len() < 150].index, inplace=True) # Catching any lyrics that are less than 120 characters

In [13]:
df.drop(df.loc[df['artist'] == 'BLACKPINK'].index, inplace=True) # dropping Blackpink, although they do self announce the lyrics are mostly Korean

### String Cleanup
I need to clean up the characters in the lyrics columns. Where there was a line break we now have \n. There are also verse sections marked [Verse], [Chorus] [Intro] and so on that need to be removed.

In [14]:
df['lyrics'][11] # before

"[Intro]\nDa-da-da-da-da-da\nDa-da-da-da-da-da\nDa-da-da-da-da-da\nDa-da-da-da-da-da (Jason Derulo)\n\n[Verse 1]\nPull up, skrrt-skrrt on your body\nPerformin' just like my 'Rari\nYou're too fine, need\u2005a\u2005ticket\nI bet you\u2005taste expensive\nPourin' up, up, up by\u2005the liter\nIf you keepin' up, you's a keeper\nTequila and vodka\nGirl, you might be a problem\n\n[Pre-Chorus]\nRun away, run away, run away, run away, I know that I should\nBut my heart wanna stay, wanna stay, wanna stay, wanna stay now\nYou can see it in my eyes that I wanna take you down right now if I could\nSo I hope you know what I mean when I say\n\n[Chorus]\nLet me take you dancin'\nTwo-step to the bedroom\nWe don't need no dancefloor\nLet me see your best move\nAnything could happen\nEver since I met you\nNo need to imagine\nBaby, all I'm asking\nIs let me take you dancing\n\n[Post-Chorus]\nLike da-da-da-da-da-da\nDa-da-da-da-da-da\nDa-da-da-da-da-da\nDa-da-da-da-da-da\nDa-da-da-da-da-da\nDa-da-da-da-d

In [15]:
df['lyrics'] = df['lyrics'].str.encode('ascii', 'ignore') # unicode present \u2005 and \u205f
df['lyrics'] = df['lyrics'].str.decode("utf-8") #used these 2 lines to encode to ascii then decode to utf-8 to remove them

df['lyrics'] = df['lyrics'].str.replace('\n', ' ').str.replace("\'", "") # removing \n and \
df['lyrics'] = df['lyrics'].str.replace(r'[\[].*?[\]]','') # removing [chorus] [intro] etc

df['lyrics'] = df['lyrics'].str.strip() #removing extra space at beginning and end

In [16]:
df['lyrics'][11] # after

'Da-da-da-da-da-da Da-da-da-da-da-da Da-da-da-da-da-da Da-da-da-da-da-da (Jason Derulo)   Pull up, skrrt-skrrt on your body Performin just like my Rari Youre too fine, needaticket I bet youtaste expensive Pourin up, up, up bythe liter If you keepin up, yous a keeper Tequila and vodka Girl, you might be a problem   Run away, run away, run away, run away, I know that I should But my heart wanna stay, wanna stay, wanna stay, wanna stay now You can see it in my eyes that I wanna take you down right now if I could So I hope you know what I mean when I say   Let me take you dancin Two-step to the bedroom We dont need no dancefloor Let me see your best move Anything could happen Ever since I met you No need to imagine Baby, all Im asking Is let me take you dancing   Like da-da-da-da-da-da Da-da-da-da-da-da Da-da-da-da-da-da Da-da-da-da-da-da Da-da-da-da-da-da Da-da-da-da-da-da Da-da-da-da-da-da Da-da-da-da-da-da   Pull up, skrrt-skrrt on your body Its just us two in this party That Louis, tha

### Featured column
Currently the featured column has html if there were featured artist and [] if no additional artists were on the track. I am going to extract the names from the tags and refill the column where possible. Although the exhisting values looks like it is a dictionary, it is actually string type.

In [17]:
df.reset_index(inplace=True, drop=True)# reseting index, since we dropped a few rows

In [18]:
feat = []    # empty featured list
for values in df['featured']: # for each item in the featured column
    if values != '[]': # if the row value is not []
        search = re.findall(r'name(.+?),', values, re.I) # search for the word name and extract the values after all the way to the comma
        res = str(search) # find all makes values list, changed it to string
        feat.append(re.sub(r'([][:\'\"])', '', res).replace(',', ' & ').strip()) # append to feat list, remove extra characters, separate artists with &, strip extra white space
#         print(res.sub(r'[^a-zA-Z0-9$]', "", res).strip('  '))
    else:
        feat.append('NA') # everything else gets appended as an empty string

# Thanks Brett & Alex for Regex Help & John for Workarounds to a dictionary that was not a dictionary.

In [19]:
df['featured'] = feat # overwriting the column with list values

### Media column
Similar to above the media column gives links/id for media of the song : Youtube, Spotify, Soundcloud. I will extract the Spotify track id, which can later be used to extract additional information about the track should I choose to.

In [20]:
track_id = []    # empty track_id list
for values in df['media']: # for each item in the media column
    if values != '[]': # if value is not []
        search = re.findall(r"track:(.+?)'", values, re.I) # find all values that have track and extract everything after, to the '
        res = str(search) # make string instead of list
        track_id.append(re.sub(r"([]\'[])", '', res).strip()) # strip extra characters and space and append

    else:
        track_id.append('NA') # otherwise append empty string
        
        
df.insert(3, 'track_id', track_id) # making a new column and placing it at 3rd column index.
df.drop('media', axis = 1, inplace = True) # dropping the old column

#https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns?page=1&tab=votes#tab-top


In [21]:
df.head()

,artist,featured,title,track_id,release_year,lyrics
0,Jason Derulo,Ty Dolla $ign & Nicki Minaj,Swalla,6kex4EBAj0WHXDKZMEJaaF,2017,Drank Young Money Love in a thousand differe...
1,Jason Derulo,2 Chainz,Talk Dirty,5aHCHpekZ854JMVyPaOt7w,2013,"(Jason, haha Jason Derulo) Haha, get Jazzy on ..."
2,Jason Derulo,Snoop Dogg,Wiggle,2sLwPnIP3CUVmIuHranJZU,2014,"Ayo, Jason (Oh yeah!) Say somethin to her, hol..."
3,Jason Derulo,NA,Trumpets,5KONnBIQ9LqCxyeSPin26k,2013,Every time that you get undressed I hear symph...
4,Jason Derulo,French Montana,Tip Toe,2z4pcBLQXF2BXKFvd0BuB6,2017,Derulo Whine fa me darlin Way you move ya spin...


### Target Column
I need to make a traget column to help determine if I would skip the track or not. I made a function to create an alias column where if the artist or featured artist has an alias it will be listed and used to check if the artists mention themselves in the lyrics

In [22]:
df.insert(2, 'aliases', '') # create a new  empty column 

In [23]:
def add_aliases(artist, aliases): # function takes artist name and alias
    df.loc[(df['artist'] == artist) | (df['featured'].str.contains(artist)), 'aliases'] += aliases # using .loc changing the value of alias

In [24]:
add_aliases('Pitbull', 'Mr. Worldwide, Mr. 305,')
add_aliases('The Notorious B.I.G.', 'Notorious,')
add_aliases('Sean Paul', 'Sean da Paul, Sean, Paul,')
add_aliases('Akon', ' Konvict,')
add_aliases('Ludacris', 'Luda,')
add_aliases('Lil Wayne', 'Weezy, Lil, Wayne,')
add_aliases('Nicki Minaj', 'Young Money, Barbie, Nicki, Minaj,')
add_aliases('Cardi B', 'Bardi, Cardi,')
add_aliases('Yasiin Bey', 'Mos Def, Mos, Def, Yasiin, Bey,')
add_aliases('Rage Against the Machine', 'Rage, Rage Against, The Machine,')
add_aliases('Run The Jewels', 'Run Them, The Jewels,')
add_aliases('“Weird Al” Yankovic', 'Weird Al, Yankovic,')
add_aliases('24kGoldn','24-karat,')
add_aliases('Pusha T', 'Pusha,')
add_aliases('Tenacious D', 'Tenacious,')
add_aliases('Jennifer Lopez', 'JLO, Jennifer, Lopez, Jenny,')
add_aliases('J Balvin', 'Balvin,')


Initially I had split the names by the space to match in the lyrics, which works for Jason Derulo (sometimes he sings the entire name, sometimes Jason and sometimes Derulo), however this caused an issue for bands that had 'the' in their name (The Beatles, The Weeknd, Run the Jewels) or like twenty one Pilots the word one.. so the skip classification became incorrect. 
I decided to just use the function and enter the aliases (not ideal), however this gave me better control of where the split should be and repetitions.

In [25]:
add_aliases('Jason Derulo', 'Jason, Derulo,')
add_aliases('Britney Spears', 'Britney, Spears,')
add_aliases('Lady Gaga', 'Lady, Gaga,')
add_aliases('Dua Lipa', 'Dua, Lipa, ')
add_aliases('Miley Cyrus', 'Miley, Cyrus,')
add_aliases('Post Malone', 'Post, Malone,')
add_aliases('Ini Kamoze', 'Ini, Kamoze,')
add_aliases('Rachel Platten', 'Rachel, Platten,')
add_aliases('Christina Perri', 'Christina, Perri,')
add_aliases('Florida Georgia Line', 'Florida Georgia, Georgia Line,')
add_aliases('Brad Paisley', 'Brad, Paisley,')        
add_aliases('Thomas Rhett', 'Thomas, Rhett,')
add_aliases('Spencer Crandall', 'Spencer, Crandall,')
add_aliases('Christina Perri', 'Christina, Perri,')
add_aliases('Christina Perri', 'Christina, Perri,')
add_aliases('Natasha Bedingfield', 'Natasha, Bedingfield,')
add_aliases('Daddy Yankee', 'Daddy, Yankee,')
add_aliases('Demi Lovato', 'Demi, Lovato,')
add_aliases('Major Lazer', 'Major, Lazer,')
add_aliases('Imagine Dragons', 'Imagine, Dragons,')
add_aliases('Matchbox Twenty', 'Matchbox, Twenty,')
add_aliases('Uncle Kracker', 'Uncle, Kracker,')
add_aliases('John, Newman', 'John, Newman,')
add_aliases('Bruno Mars', 'Bruno, Mars,')
add_aliases('Lana Del Rey', 'Lana, Del Rey,')
add_aliases('A$AP Ferg', 'A$AP, Ferg,')
add_aliases('Pharrell Williams', 'Pharrell, Williams,')
add_aliases('Billie Eilish', 'Billie, Eilish,')
add_aliases('Ariana Grande', 'Ariana, Grande, Ari,')
add_aliases('Megan Thee Stallion', 'Megan, Thee Stallion, Stallion,')
add_aliases('Doja Cat', 'Doja, Cat,')
add_aliases('Rick Astley', 'Rick, Astley,')
add_aliases('Shawn Mendes', 'Shawn, Mendes,')
add_aliases('Justin Bieber', 'Justin, Bieber,')
add_aliases('Katy Perry', 'Katy, Perry,')
add_aliases('Iggy Azalea', 'Iggy, Azalea,')
add_aliases('Lil Eazzyy', 'Lil, Eazzyy,')
add_aliases('Moneybagg Yo', 'Moneybagg,')
add_aliases('Chiddy Bang', 'Chiddy, Bang,')
add_aliases('Snoop Dogg', 'Snoop, Dogg,')
add_aliases('Missy Elliott', 'Missy, Elliott,')
add_aliases('Meek Mill', 'Meek, Mill,')
add_aliases('Lil Pump', 'Lil, Pump,')
add_aliases('Ellie Goulding', 'Ellie, Goulding,')
add_aliases('Colbie Caillat', 'Colbie, Caillat,')
add_aliases('Flo Rida', 'Flo, Rida,')
add_aliases('Gucci Mane', 'Gucci, Mane,')
add_aliases('Yo Gotti', 'Gotti,')
add_aliases('Foo Fighters', 'Foo, Fighters,')
add_aliases('Princess Nokia', 'Princess, Nokia,')
add_aliases('French Montana', 'French, Montana,')
add_aliases('Backstreet Boys', 'Backstreet, Boys,')
add_aliases('Spice Girls', 'Spice, Girls,')
add_aliases('Soulja Boy', 'Soulja,')
add_aliases('Travis Scott', 'Travis, Scott,')
add_aliases('Sam Smith', 'Sam, Smith,')
add_aliases('DJ Khaled', 'DJ, Khaled,')
add_aliases('Stone Temple Pilots', 'Stone Temple, Temple Pilots,')
add_aliases('Big Sean', 'Big, Sean,')
add_aliases('Clean Bandit', 'Clean, Bandit,')
add_aliases('Kane Brown', 'Kane, Brown,')
add_aliases('Juice WRLD', 'Juice, WRLD,')
add_aliases('Ava Max', 'Ava, Max,')
add_aliases('Taylor Swift', 'Taylor, Swift,')
add_aliases('21 Savage', '21, Savage,')

In [26]:
df.loc[df['aliases'] == '', 'aliases'] = 'NA'
df['aliases'] = df['aliases'].str.strip()

#### Now to create the Target Colum
If the artist name or the featured artist name or the alias names are in the lyrics, then it will be classified as a skip, otherwise not. I will now remove the extra commas in the alias column I added when creating the alias (was there just incase additional names were added) if I don't remove the , when I split on the , to check if the name is in the lyrics, then it will also check for spaces.

In [27]:
df['aliases'] = df['aliases'].str.replace(r',\s*$', '') # remove commas at end of aliases and any whitespace there mayy be.

In [28]:
df

,artist,featured,aliases,title,track_id,release_year,lyrics
0,Jason Derulo,Ty Dolla $ign & Nicki Minaj,"Young Money, Barbie, Nicki, Minaj,Jason, Derulo",Swalla,6kex4EBAj0WHXDKZMEJaaF,2017,Drank Young Money Love in a thousand differe...
1,Jason Derulo,2 Chainz,"Jason, Derulo",Talk Dirty,5aHCHpekZ854JMVyPaOt7w,2013,"(Jason, haha Jason Derulo) Haha, get Jazzy on ..."
2,Jason Derulo,Snoop Dogg,"Jason, Derulo,Snoop, Dogg",Wiggle,2sLwPnIP3CUVmIuHranJZU,2014,"Ayo, Jason (Oh yeah!) Say somethin to her, hol..."
3,Jason Derulo,NA,"Jason, Derulo",Trumpets,5KONnBIQ9LqCxyeSPin26k,2013,Every time that you get undressed I hear symph...
4,Jason Derulo,French Montana,"Jason, Derulo,French, Montana",Tip Toe,2z4pcBLQXF2BXKFvd0BuB6,2017,Derulo Whine fa me darlin Way you move ya spin...
...,...,...,...,...,...,...,...
11450,21 Savage,NA,"21, Savage",Hold Up,,2018,"21, 21 21, 21 Good head make a nigga toes curl..."
11451,21 Savage,NA,"21, Savage",Hollow Tips (Freestyle),,2016,I got a lot of extended clips Lot of extended ...
11452,21 Savage,NA,"21, Savage",Act A Fool,NA,NaN,"Wheezy Beats Internet, internet Fuck the inter..."
11453,21 Savage,NA,"21, Savage",Pass Her,,NaN,"Good job, First All the bitches want me (21)..."


In [29]:
skip = [] # empty list
for item in df.values: #for each row in the dataframe
    # if statement to check if name of artist/featured artist or their aliases are in the lyrics
    if (item[0].lower().strip() in item[6].lower()) or any(feat.lower().strip() in item[6] for feat in item[1].split('&') if feat.lower() != 'na') or any(alias.lower().strip()in item[6].lower() for alias in item[2].split(',') if alias.lower() != 'na'):
        skip.append(1) # append 1 if True
    else:
        skip.append(0) # append 0 if False
        


In [30]:
df['skip'] = skip

In [31]:
df['skip'].value_counts()

0    8705
1    2750
Name: skip, dtype: int64

In [32]:
df.loc[(df['track_id'] == ''), 'track_id'] = 'NA' # for concistency
df.loc[(df['track_id']=='NA')].shape

(7301, 8)

### Spotify API
I'm going to use the Spotify API to fill some missing values, like the year. Using Spotify's search ability I will look for the artist name and title and find the release year, if it is explicit and the track_id. The more important part for me is the release year, the song explicity is a bonus and track_id would just be nice to have. None of these are needed for my models, but insighful for EDA 

In [33]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import spotipy.util as util

In [34]:
df.insert(5, 'explicit', '')

In [35]:
token = util.prompt_for_user_token(#USER-ID, # my user id on spotify
                                   client_id='ID', # my 'app' information
                                   client_secret='SECRET',
                                   redirect_uri='URI')
sp = spotipy.Spotify(auth=token) # getting the authorization token

In [36]:
for song in df.values: 
    results = sp.search(song[0]+' '+song[3])
    try:
        df.loc[(df['artist'] == song[0]) & (df['title'] == song[3]), ['track_id', 'release_year', 'explicit']] = [results['tracks']['items'][0]['id'], results['tracks']['items'][0]['album']['release_date'][:4], results['tracks']['items'][0]['explicit']]
    except:
        continue

#### Filiing release year
With this Spotify filled in 1000 null values in the release year leaving 768 to be filled. To do so I will impose the median release year for each artist. Short from searching for 700 tracks on Google, it is a good alternative. There were also 7k empty values in track_id which I am now down to almost 2k. I will not be filling these.

In [37]:
df2 = df[['artist', 'release_year']] # making a new df of just artist name and release year

In [38]:
df2 = df2.loc[df['release_year'].notnull()] # only want the not null values to get the median

In [39]:
df2['release_year'] = df2['release_year'].astype(int) #need a number value for .median to work was string

In [40]:
def fill_year(artist): # function to fill release_year
    # locate artist with release year as null, fill with median year value of df2 dataframe
    df.loc[(df['release_year'].isnull()) & (df['artist'] == artist), 'release_year'] = df2.loc[df['artist'] == artist].median()[0] # there are 767 null values left in release year

In [41]:
# to fill the null values I will get the unique artist names from the null finds and iterate through, so I don't have to fill the 
# names manually
for person in df.loc[(df['release_year'].isnull())]['artist'].unique():
    fill_year(person)

In [44]:
#df.to_csv('./data/lyrics_cleaned.csv', index = False)

To the next notebook for EDA